In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mulug\\MLOPs\\MLOPs\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\mulug\\MLOPs\\MLOPs'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    min_samples_split: int
    target_column: str

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            target_column=schema.name
        )

        return model_trainer_config

In [ ]:
import pandas as pd
import os
import json
from mlProject import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]].values.ravel()
        test_y = test_data[[self.config.target_column]].values.ravel()
        
        logger.info(f"Training data shape: {train_x.shape}")
        logger.info(f"Training data features: {list(train_x.columns)}")
        logger.info(f"Target column: {self.config.target_column}")
        
        # Calculate feature statistics for validation
        feature_stats = {}
        for col in train_x.columns:
            feature_stats[col] = {
                'mean': float(train_x[col].mean()),
                'std': float(train_x[col].std()),
                'min': float(train_x[col].min()),
                'max': float(train_x[col].max())
            }
        
        # Save feature metadata
        metadata = {
            'feature_names': list(train_x.columns),
            'feature_stats': feature_stats,
            'model_config': {
                'model_type': 'RandomForest',
                'n_estimators': self.config.n_estimators,
                'max_depth': self.config.max_depth,
                'min_samples_split': self.config.min_samples_split,
                'target_column': self.config.target_column
            }
        }
        
        metadata_path = 'artifacts/feature_metadata.json'
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=4)
        logger.info(f"Saved feature metadata to {metadata_path}")
        
        # Train RandomForest model
        model = RandomForestRegressor(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            min_samples_split=self.config.min_samples_split,
            random_state=42,
            n_jobs=-1
        )
        model.fit(train_x, train_y)
        
        logger.info(f"Model trained with {self.config.n_estimators} trees")

        joblib.dump(model, os.path.join(
            self.config.root_dir, self.config.model_name))
        logger.info(f"Model saved to {os.path.join(self.config.root_dir, self.config.model_name)}")

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-09-10 10:47:11,040: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-10 10:47:11,044: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-10 10:47:11,048: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-10 10:47:11,050: INFO: common: created directory at: artifacts]
[2025-09-10 10:47:11,052: INFO: common: created directory at: artifacts/model_trainer]
[2025-09-10 10:47:11,078: INFO: 3943564012: Training data x:    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            8.0              0.60         0.22             2.1      0.080   
1            8.9              0.31         0.36             2.6      0.056   
2            8.8              0.60         0.29             2.2      0.098   
3            8.3              0.33         0.42             2.3      0.070   
4            7.5              0.60         0.03             1.8      0.095   

   free sulfur dioxide  total sulfur dioxide  densi